In [53]:
import torch
import pandas as pd
from sklearn import datasets
from collections import Counter


# Decision Tree Model

In [169]:

class Node():
    def __init__(self, feature=None, threshold=None, data_left=None, data_right=None, gain=None, value=None):
        self.feature = feature
        self.threshold = threshold
        self.data_left = data_left
        self.data_right = data_right
        self.gain = gain
        self.value = value


class DecisionTree():
    def __init__(self, min_sample=2, max_depth=5, impurity_function=None):
        self.min_sample = min_sample
        self.max_depth = max_depth
        self.impurity_function = impurity_function
        self.root = None

    def _entropy(self, y):
        _, counts = torch.unique(y, return_counts=True)
        probs = counts / counts.sum()
        return -(probs * torch.log2(probs)).sum()

    def _information_gain(self, y, y_left, y_right):
        p = y_left.shape[0] / y.shape[0]
        return self._entropy(y) - p * self._entropy(y_left) - (1 - p) * self._entropy(y_right)

    def _gini(self, y):
        _, counts = torch.unique(y, return_counts=True)
        probs = counts / counts.sum()
        return 1 - (probs ** 2).sum()

    def _gini_gain(self, y, y_left, y_right):
        p = y_left.shape[0] / y.shape[0]
        return self._gini(y) - p * self._gini(y_left) - (1 - p) * self._gini(y_right)

    def _calc_gain(self, y, y_left, y_right):
        if self.impurity_function == "gini":
            return self._gini_gain(y, y_left, y_right)
        elif self.impurity_function == "entropy":
            return self._information_gain(y, y_left, y_right)

    def _best_split(self, X, y):
        best_gain = -1
        best_split = {}

        for feature in range(X.shape[1]):
            X_crr = X[:, feature]

            for threshold in torch.unique(X_crr):
                df = torch.concat((X, y.reshape(1, -1).T), dim=1)
                df_left = df[df[:, feature] <= threshold]
                df_right = df[df[:, feature] > threshold]
                if len(df_left) > 0 and len(df_right) > 0:
                    y = df[:, -1]
                    gain = self._calc_gain(y, df_left[:, -1], df_right[:, -1])
                    if gain > best_gain:
                        best_gain = gain
                        best_split = {"feature": feature,
                                      "threshold": threshold,
                                      "data_left": df_left,
                                      "data_right": df_right,
                                      "gain": gain}
        return best_split

    def _build(self, X, y, depth=0):
        if X.shape[0] >= self.min_sample and depth <= self.max_depth:
            best = self._best_split(X, y)
            try:
                if best['gain'] > 0:
                    left = self._build(
                        best["data_left"][:, :-1], best["data_left"][:, -1], depth + 1)
                    right = self._build(
                        best["data_right"][:, :-1], best["data_right"][:, -1], depth + 1)
                    return Node(feature=best["feature"],
                                threshold=best["threshold"],
                                data_left=left,
                                data_right=right,
                                gain=best["gain"])
            except:
                pass
        return Node(
            value=Counter(y).most_common(1)[0][0]
        )

    def fit(self, X, y):
        X = torch.tensor(X)
        y = torch.tensor(y)
        self.root = self._build(X, y)

    def _predict(self, X, tree):
        # return leaf value if we are at a leaf node
        if tree.value is not None:
            return tree.value
        # traverse the tree
        feature = tree.feature
        threshold = tree.threshold
        if X[feature] < threshold:
            # go left
            return self._predict(X, tree.data_left)
        # go right
        return self._predict(X, tree.data_right)

    def predict(self, X):
        return torch.tensor([self._predict(x, self.root) for x in X])

    def print_tree(self, current_node, list_feature, nameattr='feature', left_child='data_left', right_child='data_right', indent='', last='updown'):

        if hasattr(current_node, str(nameattr)):
            def name(node): return list_feature[getattr(node, nameattr)] + ", " + str(round(node.threshold.item(), 2)) if getattr(node, nameattr) is not None else getattr(node, "value")
        else:
            def name(node): return str(node)

        up = getattr(current_node, left_child)
        down = getattr(current_node, right_child)

        if up is not None:
            next_last = 'up'
            next_indent = '{0}{1}{2}'.format(
                indent, ' ' if 'up' in last else '|', ' ' * len(str(name(current_node))))
            self.print_tree(up, list_feature, nameattr, left_child,
                    right_child, next_indent, next_last)

        if last == 'up':
            start_shape = '┌'
        elif last == 'down':
            start_shape = '└'
        elif last == 'updown':
            start_shape = ' '
        else:
            start_shape = '├'

        if up is not None and down is not None:
            end_shape = '┤'
        elif up:
            end_shape = '┘'
        elif down:
            end_shape = '┐'
        else:
            end_shape = ''

        print('{0}{1}{2}{3}'.format(
            indent, start_shape, name(current_node), end_shape))

        if down is not None:
            next_last = 'down'
            next_indent = '{0}{1}{2}'.format(
                indent, ' ' if 'down' in last else '|', ' ' * len(str(name(current_node))))
            self.print_tree(down, list_feature, nameattr, left_child,
                    right_child, next_indent, next_last)


# Adaboost

In [190]:
class AdaBoost():
    def __init__(self, X, y, num_stump = 100):
        self.num_stump = num_stump
        self.stump_list = []
        self.alpha_list = []
        self.training_error_list = []
        self.X = torch.clone(X)
        self.y = torch.clone(y)
        self.w_i = torch.ones(len(y), dtype=torch.float32)  / len(y)
        
    def _compute_error(self, y, y_pred):
        return (torch.sum(self.w_i * (torch.not_equal(y, y_pred)).int())) / torch.sum(self.w_i)
    
    def _compute_alpha(self, error):
        return 0.5 * torch.log((1 - error) / error)
    
    def _update_w(self, w_i, alpha, y, y_pred):
        return w_i * torch.exp(alpha * (torch.not_equal(y, y_pred)).int())
    
    def fit(self):
        for m in range(self.num_stump):            
            # find a weak classifier and predict label
            stump = DecisionTree(max_depth = 1, impurity_function = "entropy")
            stump.fit(self.X, self.y)
            y_pred = torch.tensor(stump.predict(self.X))
            
            self.stump_list.append(stump) # Save the model to the list

            # compute the error
            error = self._compute_error(self.y, y_pred)
            self.training_error_list.append(error)

            # compute alpha - amount of say of each stump
            alpha = self._compute_alpha(error)
            self.alpha_list.append(alpha)

            # update the weight
            self.w_i = self._update_w(self.w_i, alpha, self.y, y_pred)

            # update dataset for next stump pay more attention to the misclassified data
            idx = torch.multinomial(self.w_i, len(self.y), replacement=True)
            self.X = self.X[idx]
            self.y = self.y[idx]

        assert len(self.stump_list) == len(self.alpha_list) == len(self.training_error_list)
    
    def predict(self, X):
        X = torch.tensor(X)
        y_pred = torch.zeros(len(X), dtype=torch.float32)
        for i in range(len(self.stump_list)):
            y_pred += self.alpha_list[i] * self.stump_list[i].predict(X)
        y_pred = torch.sign(y_pred)
        y_pred[y_pred < 0] = 0
        return y_pred

# Titanic Dataset

In [162]:
path_csv = "data/titanic_modified_dataset.csv"
titanic_data_df = pd.read_csv(path_csv, index_col="PassengerId")
titanic_data_df

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,Survived
PassengerId,,,,,,,,,
1,3,0,22.0,1,0,7.2500,0,0,0
2,1,1,38.0,1,0,71.2833,1,1,1
3,3,1,26.0,0,0,7.9250,0,2,1
4,1,1,35.0,1,0,53.1000,0,1,1
5,3,0,35.0,0,0,8.0500,0,0,0
...,...,...,...,...,...,...,...,...,...
887,2,0,27.0,0,0,13.0000,0,5,0
888,1,1,19.0,0,0,30.0000,0,2,1
889,3,1,28.0,1,2,23.4500,0,2,0


In [163]:
# convert to numpy array
titanic_data_arr = torch.tensor(titanic_data_df.values, dtype=torch.float32)

# devide features to X, label to y 
X, y = titanic_data_arr[:, :-1], titanic_data_arr[:, -1]
print(X.shape)
print(y.shape)

torch.Size([891, 8])
torch.Size([891])


In [164]:
TRAIN_SIZE = 0.7
VAL_SIZE = 0.2
#TEST_SIZE = 0.1
TRAIN_SAMPLES = int(TRAIN_SIZE * titanic_data_arr.shape[0])
VAL_SAMPLES = TRAIN_SAMPLES + int(VAL_SIZE * titanic_data_arr.shape[0])

X_train, y_train = X[:TRAIN_SAMPLES], y[:TRAIN_SAMPLES]
X_val, y_val = X[TRAIN_SAMPLES:VAL_SAMPLES], y[TRAIN_SAMPLES:VAL_SAMPLES]
X_test, y_test = X[VAL_SAMPLES:], y[VAL_SAMPLES:]

print(X_train.shape, x_val.shape, X_test.shape)
print(y_train.shape, y_val.shape, y_test.shape)

torch.Size([623, 8]) torch.Size([178, 8]) torch.Size([90, 8])
torch.Size([623]) torch.Size([178]) torch.Size([90])


In [191]:
AdaBoostModel = AdaBoost(X_train, y_train, num_stump=100)
AdaBoostModel.fit()
y_preds = AdaBoostModel.predict(X_train)

C:\Users\Tran Nguyen Nhut Duy\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:86: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
C:\Users\Tran Nguyen Nhut Duy\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:87: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
C:\Users\Tran Nguyen Nhut Duy\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
C:\Users\Tran Nguyen Nhut Duy\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:48: User

In [189]:
for i in range(10):
    AdaBoostModel.stump_list[i].print_tree(AdaBoostModel.stump_list[i].root, list_feature= titanic_data_df.columns[:-1])

                   ┌0.0
         ┌Age, 12.0┤
         |         └0.0
 Sex, 0.0┤
         |           ┌1.0
         └Pclass, 2.0┤
                     └1.0
                       ┌1.0
           ┌Pclass, 1.0┤
           |           └0.0
 Title, 0.0┤
           |           ┌1.0
           └Pclass, 2.0┤
                       └1.0
                       ┌1.0
           ┌Pclass, 1.0┤
           |           └0.0
 Title, 0.0┤
           |           ┌1.0
           └Pclass, 2.0┤
                       └0.0
                        ┌0.0
           ┌Fare, 106.43┤
           |            └1.0
 Title, 0.0┤
           |           ┌0.0
           └Pclass, 2.0┤
                       └0.0
                       ┌0.0
           ┌Fare, 83.47┤
           |           └0.0
 Title, 0.0┤
           |          ┌1.0
           └SibSp, 2.0┤
                      └1.0
                       ┌0.0
           ┌Fare, 83.47┤
           |           └1.0
 Title, 0.0┤
           |          ┌1.0
           └SibSp, 2.0┤


In [192]:
y_preds = AdaBoostModel.predict(X_val)
print(y_preds)
print("Accuracy: ", torch.sum(y_preds == y_val) / len(y_val))

C:\Users\Tran Nguyen Nhut Duy\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:48: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
Accuracy:  tensor(0.6629)


# Iris Dataset

In [193]:

iris = datasets.load_iris()
iris_df = pd.DataFrame(iris.data, columns = ["sepal length (cm)", "sepal width (cm)", "petal length (cm)", "petal width (cm)"])
iris_df['target'] = iris.target
list_feature = ["sepal length", "sepal width", "petal length", "petal width"]
iris_df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [194]:
# convert to numpy array
titanic_data_arr = torch.tensor(titanic_data_df.values, dtype=torch.float32)

# devide features to X, label to y 
X, y = titanic_data_arr[:, :-1], titanic_data_arr[:, -1]
print(X.shape)
print(y.shape)

torch.Size([891, 8])
torch.Size([891])


In [195]:
TRAIN_SIZE = 0.7
VAL_SIZE = 0.2
#TEST_SIZE = 0.1
TRAIN_SAMPLES = int(TRAIN_SIZE * titanic_data_arr.shape[0])
VAL_SAMPLES = TRAIN_SAMPLES + int(VAL_SIZE * titanic_data_arr.shape[0])

X_train, y_train = X[:TRAIN_SAMPLES], y[:TRAIN_SAMPLES]
X_val, y_val = X[TRAIN_SAMPLES:VAL_SAMPLES], y[TRAIN_SAMPLES:VAL_SAMPLES]
X_test, y_test = X[VAL_SAMPLES:], y[VAL_SAMPLES:]

print(X_train.shape, x_val.shape, X_test.shape)
print(y_train.shape, y_val.shape, y_test.shape)

torch.Size([623, 8]) torch.Size([178, 8]) torch.Size([90, 8])
torch.Size([623]) torch.Size([178]) torch.Size([90])


In [196]:
AdaBoostModel = AdaBoost(X_train, y_train, num_stump=100)
AdaBoostModel.fit()
y_preds = AdaBoostModel.predict(X_train)
print("Accuracy: ", torch.sum(y_preds == y_train) / len(y_train))

C:\Users\Tran Nguyen Nhut Duy\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:86: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
C:\Users\Tran Nguyen Nhut Duy\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:87: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
C:\Users\Tran Nguyen Nhut Duy\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
C:\Users\Tran Nguyen Nhut Duy\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:48: User

Accuracy:  tensor(0.6051)


In [197]:
y_preds = AdaBoostModel.predict(X_val)
print(y_preds)
print("Accuracy: ", torch.sum(y_preds == y_val) / len(y_val))

C:\Users\Tran Nguyen Nhut Duy\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:48: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
Accuracy:  tensor(0.6517)
